# **Deep Zork**
***
Can a machine learn to play a text-based game?

<img src='zork_box_art.jpg'/>

## Project Motivation
***
Advances in machine learning involving reinforcement learning has seen the rise of AIs capable of playing video games and outperforming human players. These include all sorts of games from chess and Go to Atari games. All of these instances involve games in which gameplay is based on visuals. Using convolutional neural networks, machines become quite capable of recognizing patterns in changing pixels on a screen and can react in real-time in order to win. 

Enter text-based games.

Also known as interactive fiction games, text-based games were some of the first ever computer games due to relying solely on text for gameplay and not needing to process graphics. They rely on language parsing text commands given by the player in response to changing descriptions of the player's environment. 

This presents a whole different type of problem for AIs attempting to play one of these games. In text-based games, reaction time is not a factor. Instead, the player must have an understanding of the language used and to be able to make smart decisions based off of subtle hints in the game's descriptions.

For example, if a game tells the player that, to the north lies a monster that will kill the player if they head that direction, the player should know not to go that way.

There have been some attempts at teaching machines to play text-based games with a key example being __TextWorld__ by Microsoft with variable success. There is still, however, a long way to go before an interactive fiction AI will perform at the level of AlphaGo or AlphaZero.

This project will aide in tackling the issue by focusing on teaching a machine to play one single game - Zork - in hopes that the techniques used can be applied to other games in the future. 


## About Zork
***
### History
***
One of the most classic and well-known text-based game, Zork I (also known as Zork: The Great Underground Empire), was written in the late 1970s. At the time it was well-revered for its compelling story-telling and also its language parsing engines which allowed players to use not only simple commands ('Attack monster') but more complex ones with prepositions ('Attack the monster with the sword'). It was one of the most successful works of interactive fiction and its legacy is still alive today with many elements still being used in newer works

### Plot
***
Zork is first and foremost an adventure game. The ultimate goal of the game is to collect all 19 different treasures and install them in a trophy case. To do so, the player must explore all areas of the game, including a sprawling underground dungeon.

### Map
***
Below are two maps of the Zork world - above and below ground. As you can see it's a very large map with many different rooms and paths between them. Note that the player always starts at **West of House** on the above ground map.


<img src='zork_map_1.gif'/> <img src='zork_map_2.gif'/>

## Prework
***
### Commands
There are three types of commands used in Zork.

1) Basic commands - commands to go a certain direction, commands to get info
        - Go north, go south
        - Look, check inventory
2) Verb-Object commands - commands that consist of a verb and a noun or noun phrase
        - Take key, open the door
3) Verb-Object-Prep-Object commands - commands that consist of a verb and noun phrase followed by a preposition and second noun phrase
        - Attack the monster with the sword, unlock the chest with the key
    
The objects in these commands can be gathered from the descriptions in-game. The verbs however need to be provide beforehand.

### Verb dictionary

To generate a list of possible verbs used in commands, info from the game file itself was gathered using ZTools (https://www.inform-fiction.org/zmachine/ztools.html).
The text was processed using Matcher and POS from the <b>spacy</b> library to find all the possible verbs and verb phrases. This can be seen in the ZorkVerbs notebook.

This list was then manually picked through to remove commands that would not influence gameplay (commands that save the game, change the text descriptions, etc.)

Also, for the 2nd type of commands, the noun phrase was replaced with 'OBJ' and for the 3rd type of commands the first noun phrases were replaced with 'OBJ' and the second with 'DCT' to allow for easy substitution of nouns.

The commands were then combined into a <i>commands</i> class and stored in a seperate script file (game_commands.py). The lists can be seen below.

In [3]:
from game_commands import commands
cmd = commands()

In [11]:
## Basic commands
cmd.basic_actions

['go north',
 'go south',
 'go west',
 'go east',
 'go northeast',
 'go northwest',
 'go southeast',
 'go southwest',
 'd',
 'u']

In [7]:
## Verb-object commands
cmd.command1_actions

['open OBJ',
 'get OBJ',
 'set OBJ',
 'hit OBJ',
 'eat OBJ',
 'put OBJ',
 'cut OBJ',
 'dig OBJ',
 'ask OBJ',
 'fix OBJ',
 'make OBJ',
 'wear OBJ',
 'move OBJ',
 'kick OBJ',
 'kill OBJ',
 'find OBJ',
 'play OBJ',
 'feel OBJ',
 'hide OBJ',
 'read OBJ',
 'fill OBJ',
 'flip OBJ',
 'burn OBJ',
 'pick OBJ',
 'pour OBJ',
 'pull OBJ',
 'apply OBJ',
 'leave OBJ',
 'ask OBJ',
 'break OBJ',
 'enter OBJ',
 'curse OBJ',
 'shake OBJ',
 'burn OBJ',
 'inflate OBJ',
 'brandish OBJ',
 'donate OBJ',
 'squeeze OBJ',
 'attach OBJ',
 'find OBJ',
 'banish OBJ',
 'read OBJ',
 'enchant OBJ',
 'feel OBJ',
 'pour OBJ']

In [8]:
## Verb-object-prep-object commands
cmd.command2_actions

['pour OBJ on DCT',
 'hide OBJ in DCT',
 'pour OBJ in DCT',
 'move OBJ in DCT',
 'hide OBJ on DCT',
 'flip OBJ for DCT',
 'fix OBJ with DCT',
 'spray OBJ on DCT',
 'dig OBJ with DCT',
 'cut OBJ with DCT',
 'pick OBJ with DCT',
 'squeeze OBJ on DCT',
 'pour OBJ from DCT',
 'fill OBJ with DCT',
 'brandish OBJ at DCT',
 'burn OBJ with DCT',
 'flip OBJ with DCT',
 'read OBJ with DCT',
 'hide OBJ under DCT',
 'carry OBJ from DCT',
 'inflate OBJ with DCT',
 'unlock OBJ with DCT',
 'give OBJ to DCT',
 'carry OBJ to DCT',
 'spray OBJ with DCT']

### Command likelihood
***
When the nouns from the game's descriptions are substituted into the above commands, certain combinations might not make much sense. For example, if there is a door and a table in the environment, the command "cut the door with the key" would not make any sense.

To help with this problem, walkthroughs and tutorials from other text-based games can be studied to have a way to determine the relevance and likelihood of different commands.

This can all be found in the scrape_tutorials notebook but the basic idea is that tutorials and walkthroughs from two databases - https://www.ifarchive.org/indexes/if-archive/solutions/ and http://www.textfiles.com/adventure/ were scraped using __urllib__ and __BeautifulSoup__ to get text files containing relevant commands for these games.

Those text files were then clean, preprocessed, and combined into one single file. The entire corpus was then ran through NLP using __spacy__.

To determine likelihood of command phrases, a Word2Vec model was used to determine the similarity score.

An example of this usage can be seen below.

In [2]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
f = open('tutorials_2.txt', 'r')
tutorials = f.read()
sentences = word_tokenize(tutorials)
w2v = Word2Vec([sentences])

In [15]:
w2v.wv.similarity('open', 'table')

0.020616088

In [6]:
w2v.wv.similarity('open', 'chest')

0.97727084

In [12]:
w2v.wv.similarity(word_tokenize('attack chest with'), 'key').mean()

0.40543437

In [13]:
w2v.wv.similarity(word_tokenize('unlock chest with'), 'key').mean()

0.4204148

### Game emulation
***
The majority of text-based games can be played using Z-Machine data files (.z3, .z4, .z5, .z8). To play games with this file format there are several emulators for different systems. For Windows one of the best emulators is __Frotz__ (https://github.com/DavidGriffith/frotz).

Using __Popen__ in Python, a .exe version of the program can be used to play the Z-Machine games.

In order to read the printout line by line without blocking the program a seperate thread is used and the lines can be read through a queue. 

## Methodology 
***
### Reinforcement learning
***
![](rl_diagram2.jpg)



For any general reinforcement learning network, there must be an __agent__ which, given a game __state__,  chooses an __action__ which then interacts with the __enviornment__. The new __state__ along with a corresponding __reward__ are then passed back to the __agent__ which then adjusts its weights for future decisions.

### Environment
For this project, the environment is the __Zork 1__ game which connects to Python via __Popen__ in order to be able to write new commands and read the lines given by the game.

### State
The state consists of both the _surroundings_ in the game as well as the _inventory_ of the player. The combination of these two will provide all necessary information including relevant objects to interact with.

### Reward
The reward should influence "good" behaviour while penalizing "bad" behaviour by the agent. There is already an in-game score given to the player when reaching certain areas or performing certain actions. However additional rewards can help to direct the agent more successfully. 

One of the most important parts of playing a text-based game is _exploration_. Any human player playing Zork would need to take part in exploration of the various areas and rooms to learn more about the game world and to find essential items.

Another important part is finding, taking, and using _items_ in the game. For Zork especially, the main goal is to find and collect the various treasures so interacting with items should be rewarded. 

As far as negative rewards go, taking too many turns in a game usually is a bad thing as the goal should be to get to the end as quick as possible. Thus, a small negative reward can be given per each turn to help dissuade the agent from taking more turns than neccesary. 

Below are the different rewards.

__negative_per_turn_reward__ : -1 points
    - Each turn taken this reward will be added to the total reward for the round. This way, on turns where no other points were scored, the turn will be remembered as being non-productive. 
    
__new_area_reward__ : 20 points
    - Discovering a new area or room in the game should be rewarded in order to encourage exploration.
    
__moving_around_reward__: 0.5 points
    - In order to prevent the agent from staying in one room too long, a small positive reward is given each time the agent moves from one area to another that has already been visited. This also helps to encourage the agent to return to an area it previously left when it had not yet completed all objectives in that area. Providing this reward instead of the new_area_reward when the agent revisits an area will prevent it from exploiting the reward system and hopping back and forth between rooms.

__inventory_reward__ : 50 points
    - If the agent picks up an item or uses an item, the in-game inventory will change. This will be rewarded with a large number of points as finding the treasures of the game ultimately is how the agent can win. 
    
__inventory_not_new_reward__ : 0.5 points
    - In order to prevent the agent taking advantage of the large inventory_reward, a much smaller reward will be given when an inventory change occurs that has already taken place. This will prevent exploitable situations in which the agent repeatedly picks up and then drops an item.

__in_game_score_reward__ : 10 * in-game score
    - Scoring points in the game is the best type of action the agent could take so it should be rewarded the greatest. Each time an in-game score is granted, a weight (initially set at 10) will be applied to it to show its significance to the agent. 

### Agent
***
The __agent__ consists of a neural network - more specifically a Deep-Q Network (referred to as DQN). The architecture of the model can be seen below.

#### Input 
***
The model takes in the _state_ and performed _action_ as seperate inputs. The inputs are preprocessed (removing special characters, converting to lower-case) and tokenized using a keras tokenizer with a vocabulary size of 1200. The tokens are then padded to ensure a consistent length of 50.
#### Embedding
***
The _state_ and _action_ inputs are fed separately into a shared embedding layer with a dimension of 16. This layer is shared so that the vector for a word found in a _state_ will be the same vector if the word is in an _action_.
#### LSTM
***
The _state_ and _action_ embeddings are then sent to a long short-term memory or LSTM layer. This layer hopes to capture some of the time based features in the _states_ and _actions_. For example, if the state is "To the north lies a treasure chest and to the west there is a monster.", the model should be able to capture the signficance of the order of the description. The LSTM layer uses a dimension of 32.
#### Dense
*** 
The model passes the output from the LSTM layers to separate dense layers of dimension 8. The activation function used at this layer is 'tanh'.
#### Interaction
***
Lastly, to combine the dense _state_ and _action_ layers, a Dot layer is used to represent the condensed information of both inputs into a single value.


### Q-Learning
***
Q-learning allows an agent to make decisions based on future rewards. Instead of just picking the action that gives the highest reward for the current turn, Q-learning takes into account the future reward of the next state following an action as well.

Below is the formula for the Q-function.

<img src='q_function.png'/>

The new calculated Q-value of a given state is found by using the old Q-value for that state and the sum of the current reward and the highest possible reward for the next state multiplied by a discount rate. The future reward is discounted because there is uncertainty in the predicted reward. Lastly, the sum of the current and future rewards is multiplied by a learning rate which slows down the learning process.

In this project a learning rate of 0.1 and discount factor of 0.75 are used.

## Goals 
***
The following shows different checkpoints of performance of the AI in this project. These can be used to measure and judge the performance of the AI.

__Level 0__: Able to play the game at a basic level.
    - The AI can process descriptions given by the game and generate likely actions from them.

__Level 1__: Able to navigate across various areas and rooms.
    - The AI can travel and discover new areas without getting stuck in one area for too long.

__Level 2___: Able to complete mini-quests.
    - The AI can complete simple tasks such as taking the egg from up in a tree in the forest and opening and enter the window into the house.

__Level 3___: Able to access the underground.
    - The AI can explore all areas above ground and find its way to the underground.

__Level 4___: Able to collect treasures.
    - The AI can find and gather more than one treasure without losing.

__Level 5__: Able to beat the game.
    - The AI can find all the treasures and win the game.

## Experimentation
***
Run for 1000 games, 250 turns to build vocabulary and find invalid actions
Run for 50 games, 250 turns, batch size of 50
Epsilon decay of 0.99